In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
torch.cuda.set_device(3)

In [3]:
#读取res meta数据集
Res_meta = open('./YelpChi/output_meta_yelpResData_NRYRcleaned.txt')
Date = []
review_ID = []
reviewer_ID = []
product_ID = []
Label = []
Useful = []
Funny = []
Cool = []
star_rating = []
Res_data=Res_meta.readlines()
for line in Res_data:
    line=line.rstrip("\n")
    line=line.split()
    Date.append(line[0])
    review_ID.append(line[1])
    reviewer_ID.append(line[2])
    product_ID.append(line[3])
    Label.append(line[4])
    Useful.append(line[5])
    Funny.append(line[6])
    Cool.append(line[7])
    star_rating.append(int(line[8]))

In [4]:
reviewid_pd = pd.DataFrame(data = reviewer_ID,columns=['reviewerID'])
reviewid_pd['reviewerID'].describe()

count                      61541
unique                     33964
top       jrMxBHW3SlUGVGYWvPsHWA
freq                          57
Name: reviewerID, dtype: object

In [5]:
#读取res review数据集
Res_review = open('./YelpChi/output_review_yelpResData_NRYRcleaned.txt')
review = []
ff=Res_review.readlines()
for line in ff:
    line=line.rstrip("\n")
    review.append(line)

In [6]:
tok = Tokenizer(lower=True,num_words=10000)
tok.fit_on_texts(review)
index_review = tok.texts_to_sequences(review)
index_review = pad_sequences(index_review, maxlen=100,padding='post')

In [7]:
index_review = index_review.tolist()

In [8]:
reviewerID_review = {}
for i in range(len(reviewer_ID)):
    if reviewer_ID[i] not in reviewerID_review.keys():
        reviewerID_review[reviewer_ID[i]] = index_review[i]
#     elif reviewer_ID[i] in reviewerID_review.keys():
#         reviewerID_review[reviewer_ID[i]].extend(index_review[i])

In [9]:
index_review_complete_reviewer = []
for i in range(len(reviewer_ID)):
    index_review_complete_reviewer.append(reviewerID_review[reviewer_ID[i]])

In [10]:
productID_review = {}
for i in range(len(product_ID)):
    if product_ID[i] not in productID_review.keys():
        productID_review[product_ID[i]] = index_review[i]

In [11]:
index_review_complete_product = []
for i in range(len(product_ID)):
    index_review_complete_product.append(productID_review[product_ID[i]])

In [12]:
#拆分虚假评论一份，真实评论一份，完整一份，并且打乱划分7，3划分数据集
real_reviewer = []
spam_reviewer = []
real_label = []
real_product = []
spam_product = []
spam_label = []
for i in range(len(Label)):
    if Label[i] == 'N':
        real_reviewer.append(index_review_complete_reviewer[i])
        real_product.append(index_review_complete_product[i])
        real_label.append(star_rating[i])
    else:
        spam_reviewer.append(index_review_complete_reviewer[i])
        spam_product.append(index_review_complete_product[i])
        spam_label.append(star_rating[i])

In [13]:
#total数据集
X_total_Reviewer = np.array(index_review_complete_reviewer)
X_total_product = np.array(index_review_complete_product)
Y_total_label = np.array(star_rating)
Y_total_label = Y_total_label.reshape(Y_total_label.shape[0],1)
total_data = np.concatenate((X_total_Reviewer,X_total_product,Y_total_label),axis=1)
print(total_data.shape)

#spam数据集
X_spam_Reviewer = np.array(spam_reviewer)
X_spam_product = np.array(spam_product)
Y_spam_label = np.array(spam_label)
Y_spam_label = Y_spam_label.reshape(Y_spam_label.shape[0],1)
spam_data = np.concatenate((X_spam_Reviewer,X_spam_product,Y_spam_label),axis=1)
print(spam_data.shape)

#real数据集
X_real_Reviewer = np.array(real_reviewer)
X_real_product = np.array(real_product)
Y_real_label = np.array(real_label)
Y_real_label = Y_real_label.reshape(Y_real_label.shape[0],1)
real_data = np.concatenate((X_real_Reviewer,X_real_product,Y_real_label),axis=1)
print(real_data.shape)

(61541, 201)
(8141, 201)
(53400, 201)


In [14]:
#打乱全部的数据集并划分
np.random.shuffle(total_data)
total_train = total_data[:int(total_data.shape[0]*0.7)]
total_test = total_data[int(total_data.shape[0]*0.7):]
x_total_train = total_train[:,:-1]
y_total_train = total_train[:,200:]
x_total_test = total_test[:,:-1]
y_total_test = total_test[:,200:]

In [15]:
#打乱spam的数据集并划分
np.random.shuffle(spam_data)
spam_train = spam_data[:int(spam_data.shape[0]*0.7)]
spam_test = spam_data[int(spam_data.shape[0]*0.7):]
x_spam_train = spam_train[:,:-1]
y_spam_train = spam_train[:,200:]
x_spam_test = spam_test[:,:-1]
y_spam_test = spam_test[:,200:]

In [16]:
#打乱real的数据集并划分
np.random.shuffle(real_data)
real_train = real_data[:int(real_data.shape[0]*0.7)]
real_test = real_data[int(real_data.shape[0]*0.7):]
x_real_train = real_train[:,:-1]
y_real_train = real_train[:,200:]
x_real_test = real_test[:,:-1]
y_real_test = real_test[:,200:]

In [17]:
trian_total = torch.utils.data.TensorDataset(torch.from_numpy(x_total_train).long(), torch.from_numpy(y_total_train).long())#（特征，标签）
train_total_loader = torch.utils.data.DataLoader(trian_total,batch_size=128, shuffle=True)

In [18]:
spam_total = torch.utils.data.TensorDataset(torch.from_numpy(x_spam_train).long(), torch.from_numpy(y_spam_train).long())#（特征，标签）
spam_total_loader = torch.utils.data.DataLoader(spam_total,batch_size=128, shuffle=True)

In [19]:
real_total = torch.utils.data.TensorDataset(torch.from_numpy(x_real_train).long(), torch.from_numpy(y_real_train).long())#（特征，标签）
real_total_loader = torch.utils.data.DataLoader(real_total,batch_size=128, shuffle=True)

In [20]:
class Net(torch.nn.Module):
    def __init__(self,):
        super(Net, self).__init__()
        self.embedding_user = torch.nn.Embedding(10000,100)
        self.embedding_item = torch.nn.Embedding(10000,100)
        self.linear_user_1 = torch.nn.Linear(10000,100)
        self.linear_item_1 = torch.nn.Linear(10000,100)        
        self.linear_user_2 = torch.nn.Linear(100,1)
        self.linear_item_2 = torch.nn.Linear(100,1)
        
        
    def forward(self,X):
        x_user = self.embedding_user(X[:,:100])
        x_item = self.embedding_item(X[:,100:])
        x_user = x_user.reshape(x_user.shape[0],10000)
        x_item = x_item.reshape(x_item.shape[0],10000)
        x_user = torch.nn.functional.relu(self.linear_user_1(x_user))
        x_item = torch.nn.functional.relu(self.linear_item_1(x_item))
        x_user = torch.nn.functional.relu(self.linear_user_2(x_user))
        x_item = torch.nn.functional.relu(self.linear_item_2(x_item))
        predict = x_user * x_item
        return predict

In [21]:
model = Net()
model = model.to('cuda')

In [22]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) 

In [23]:
for epoch in range(30):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_total_loader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs.float(), labels.float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0
print('Finished Training')

[1,    20] loss: 7.780
[1,    40] loss: 2.393
[1,    60] loss: 1.911
[1,    80] loss: 1.775
[1,   100] loss: 1.676
[1,   120] loss: 1.887
[1,   140] loss: 1.586
[1,   160] loss: 1.599
[1,   180] loss: 1.651
[1,   200] loss: 1.699
[1,   220] loss: 1.744
[1,   240] loss: 1.546
[1,   260] loss: 1.594
[1,   280] loss: 1.603
[1,   300] loss: 1.641
[1,   320] loss: 1.659
[2,    20] loss: 1.189
[2,    40] loss: 1.212
[2,    60] loss: 1.356
[2,    80] loss: 1.282
[2,   100] loss: 1.353
[2,   120] loss: 1.205
[2,   140] loss: 1.236
[2,   160] loss: 1.072
[2,   180] loss: 1.124
[2,   200] loss: 1.156
[2,   220] loss: 1.180
[2,   240] loss: 1.245
[2,   260] loss: 1.181
[2,   280] loss: 1.182
[2,   300] loss: 1.316
[2,   320] loss: 1.228
[3,    20] loss: 0.900
[3,    40] loss: 0.935
[3,    60] loss: 0.934
[3,    80] loss: 0.852
[3,   100] loss: 0.898
[3,   120] loss: 0.966
[3,   140] loss: 0.976
[3,   160] loss: 1.008
[3,   180] loss: 1.007
[3,   200] loss: 0.994
[3,   220] loss: 1.045
[3,   240] 

[22,   260] loss: 0.615
[22,   280] loss: 0.635
[22,   300] loss: 0.618
[22,   320] loss: 0.593
[23,    20] loss: 0.488
[23,    40] loss: 0.495
[23,    60] loss: 0.509
[23,    80] loss: 0.510
[23,   100] loss: 0.532
[23,   120] loss: 0.523
[23,   140] loss: 0.543
[23,   160] loss: 0.576
[23,   180] loss: 0.556
[23,   200] loss: 0.616
[23,   220] loss: 0.629
[23,   240] loss: 0.608
[23,   260] loss: 0.529
[23,   280] loss: 0.565
[23,   300] loss: 0.592
[23,   320] loss: 0.613
[24,    20] loss: 0.490
[24,    40] loss: 0.511
[24,    60] loss: 0.526
[24,    80] loss: 0.523
[24,   100] loss: 0.534
[24,   120] loss: 0.549
[24,   140] loss: 0.507
[24,   160] loss: 0.554
[24,   180] loss: 0.562
[24,   200] loss: 0.604
[24,   220] loss: 0.595
[24,   240] loss: 0.583
[24,   260] loss: 0.554
[24,   280] loss: 0.564
[24,   300] loss: 0.642
[24,   320] loss: 0.608
[25,    20] loss: 0.489
[25,    40] loss: 0.524
[25,    60] loss: 0.496
[25,    80] loss: 0.528
[25,   100] loss: 0.534
[25,   120] loss

In [24]:
y_predict = model(torch.from_numpy(x_total_test).to('cuda'))

In [103]:
from sklearn.metrics import mean_squared_error
from math import sqrt
real_rms = sqrt(mean_squared_error(y_predict.cpu().detach().numpy(), y_real_test))
real_rms

1.1562512950548132

In [109]:
from sklearn.metrics import mean_squared_error
from math import sqrt
spam_rms = sqrt(mean_squared_error(y_predict.cpu().detach().numpy(), y_spam_test))
spam_rms

1.4975770709515006

In [25]:
from sklearn.metrics import mean_squared_error
from math import sqrt
total_rms = sqrt(mean_squared_error(y_predict.cpu().detach().numpy(), y_total_test))
total_rms

1.1845773384796605